In [ ]:
################################################################################
# MASTERARBEIT - SKRIPT 06:
# EXPERIMENT 1.5 (Forschungsfrage 1) - AUSREISSERERKENNUNG (LLM)
################################################################################
#
# ZWECK DIESES SKRIPTS (Methodik gemäß Abschnitt 3.3.1 und 3.3.4):
#
# 1. (Laden): Lädt den 'schmutzigen' Rohdatensatz (rfd_main.csv).
#
# 2. (Zielspalten/Kontext): Die Aufgabe bleibt konsistent: 
#    Identifiziere Ausreißer in den Spalten 'replies', 'views', 'votes'.
#
# 3. (Methode): Wendet die fünfte Methode (zweite moderne KI-Methode) an:
#    Ein großes Sprachmodell (LLM), namentlich Claude 4.5 Sonnet.
#
# 4. (Methodischer Ansatz - SEMANTISCHE ANALYSE):
#    Im Gegensatz zu den statistischen (IQR) oder dichte-basierten (LOF) 
#    Methoden, wird das LLM angewiesen, JEDE ZEILE als GANZEN 
#    (d.h. inklusive Textspalten wie 'title' oder 'price') als KONTEXT 
#    (Kontext) zu analysieren. 
#    Basierend auf diesem semantischen Verständnis soll das LLM bewerten, 
#    ob die Werte in den Zielspalten ('replies', 'views', 'votes') 
#    innerhalb dieses Kontexts plausibel oder anomal (Ausreißer) sind.
#
# 5. (Detektion & Begründung): Das LLM wird angewiesen, zwei Dinge 
#    zurückzugeben:
#    a) Eine binäre Entscheidung (Ja/Nein) für die Zielspalten.
#    b) Eine kurze, stichpunktartige Begründung (Begründung), WARUM 
#       es diese Entscheidung getroffen hat (entscheidend für Forschungsfrage 4).
#
# 6. (Speichern): Speichert die Ergebnisse. Um eine vollständige Analyse 
#    (quantitativ und qualitativ) zu ermöglichen, werden gespeichert:
#    a) Die identifizierten Ausreißer-Indizes.
#    b) Die vom LLM generierten Begründungen (für die qualitative Analyse).
#
################################################################################

# Schritt 1: Notwendige Bibliotheken importieren
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys 
import warnings
import json # Wichtig für die strukturierte Ein- und Ausgabe des LLM
import time  # Für die Messung der Laufzeit des LLM-Experiments

# Importieren der Anthropic-Bibliothek für die Claude-API
# (gemäß Abschnitt 3.5 'Implementierung' als konkrete Implementierung gewählt)
print("Lade notwendige Bibliotheken (Pandas, Numpy, Matplotlib, OS, sys, json)...")
try:
    from anthropic import Anthropic, RateLimitError, APIError
    print("Anthropic (Claude) Bibliothek erfolgreich geladen.")
except ImportError as e:
    print(f"FEHLER: Kritische Bibliothek (Anthropic) konnte nicht geladen werden.")
    print(f"Fehlermeldung: {e}")
    print("Stellen Sie sicher, dass 'pip install anthropic' ausgeführt wurde.")
    sys.exit("Skript gestoppt, da Abhängigkeiten fehlen.")

print("Alle Bibliotheken sind bereit.")
print("=" * 70)
#
################################################################################

# SCHRITT 2: Laden des 'schmutzigen' Rohdatensatzes und API-Konfiguration
print("--- Schritt 2: 'Schmutzigen' Rohdatensatz laden & API-Client initialisieren ---")

# --- 2.1: Laden der Daten ---
# Gemäß ZWECK-Schritt 1 wird der 'schmutzige' Rohdatensatz (rfd_main.csv) geladen.
# Dies ist methodisch zwingend, um die Konsistenz mit den 
# Experimenten 1.1 (IQR), 1.2 (LOF), 1.3 (ISO) und 1.4 (TabPFN) 
# zu wahren (alle 1326 Zeilen).
dateipfad = 'rfd_main.csv'
df_schmutzig = pd.read_csv(dateipfad)

# Die .shape-Ausgabe bestätigt die Dimensionen (1326 Zeilen).
print(f"Datensatz geladen: {dateipfad}")
print(f"Dimensionen (Zeilen, Spalten): {df_schmutzig.shape}")
print("-" * 40)

# --- 2.2: Initialisierung des API-Clients ---
# Wir müssen den 'Anthropic'-Client initialisieren, um Anfragen 
# an das Claude-Modell senden zu können.
#
# WICHTIG: Der API-Schlüssel muss in den Systemumgebungsvariablen 
# als 'ANTHROPIC_API_KEY' gespeichert sein, damit dieser Code 
# (client = Anthropic()) funktioniert.
try:
    client = Anthropic() 
    # (Der Client liest den API-Schlüssel automatisch aus der Umgebungsvariable)
    print("Anthropic (Claude) API-Client erfolgreich initialisiert.")
except Exception as e:
    print(f"FEHLER: Der Anthropic API-Client konnte nicht initialisiert werden.")
    print("Stellen Sie sicher, dass der 'ANTHROPIC_API_KEY' in Ihren \
Umgebungsvariablen gesetzt ist.")
    print(f"Fehlermeldung: {e}")
    sys.exit("Skript gestoppt, da API-Konfiguration fehlt.")
    
print("=" * 70)
#
################################################################################

# SCHRITT 3: Definition der Ziel- und Kontextspalten
print("\n--- Schritt 3: Definition der Ziel- und Kontextspalten ---")

# --- 3.1: Zielspalten (Konsistenz der Forschungsfrage 1) ---
#
# Methodische Konsistenz: Um die Ergebnisse mit IQR, LOF, TabPFN, etc. 
# vergleichen zu können (Forschungsfrage 1), bleibt die Kernaufgabe 
# des Modells identisch: Finde statistische Ausreißer in DIESEN drei Spalten.
zielspalten = ['replies', 'views', 'votes']
print(f"Zielspalten (zu bewerten): {zielspalten}")

# --- 3.2: Kontextspalten (Semantische Analyse) ---
#
# Methodischer Ansatz (gemäß ZWECK-Schritt 4):
# Hier liegt der Hauptunterschied des LLM-Experiments. Wir geben dem LLM 
# den gesamten semantischen Kontext der Zeile, damit es eine "intelligente" 
# Entscheidung treffen kann.
kontextspalten = [
    'title', 
    'price', 
    'saving', 
    'parent_category', 
    'thread_category', 
    'source',
    'author',
    # Die Zielspalten müssen auch im Kontext enthalten sein:
    'replies', 
    'views', 
    'votes'
]
print(f"Kontextspalten (zur Analyse): {len(kontextspalten)} Spalten werden \
an das LLM gesendet.")

# --- 3.3: Finale Datenvorbereitung (Umgang mit NaN im KONTEXT) ---
#
# Methodische Notwendigkeit (basierend auf EDA Skript 01):
# Die EDA (Skript 01, .describe() / .info()) hat gezeigt, dass unsere 
# ZIELspalten ('replies', 'views', 'votes') KEINE NaNs haben.
#
# ABER: Unsere KONTEXTspalten (z.B. 'price', 'saving', 'source') 
# enthalten HUNDERTE von NaN-Werten (siehe EDA-Analyse 'count').
#
# Ein 'NaN'-Wert kann von einem LLM nicht eindeutig interpretiert werden 
# (er wird oft als 'null' oder als String "NaN" gesendet). Dies würde als 
# störendes "Rauschen" im Prompt wirken.
#
# Methodische Entscheidung: Um dem LLM einen sauberen, neutralen Input 
# (z.B. "price: ''" statt "price: null") zu geben, konvertieren wir alle 
# 'NaN'-Werte aus den Kontextspalten in einen leeren String ('').
print("\nBereite LLM-Input-Daten vor (Konvertiere NaN zu leeren Strings)...")

df_llm_input = df_schmutzig[kontextspalten].copy()
df_llm_input = df_llm_input.fillna('')

# Bestätigen wir das Ergebnis (alle Spalten sollten 'non-null' sein)
print("Vorbereitung abgeschlossen. Datenstruktur für LLM-Input:")
df_llm_input.info()

print("=" * 70)
#
################################################################################

# SCHRITT 4: Definition der Prompt-Strategie (System & User Prompts)
print("\n--- Schritt 4: Definition der Prompt-Strategie ---")

# Methodischer Ansatz (gemäß ZWECK-Schritt 4 & 5):
# Wir definieren eine "Prompt-Strategie", um das LLM anzuweisen.
# Diese besteht aus zwei Teilen:
#
# 1. SYSTEM_PROMPT: Definiert die Rolle, die Aufgabe, den Kontext und 
#    das ZWINGEND erforderliche Ausgabeformat (JSON).
# 2. USER_PROMPT (Funktion): Formatiert jede einzelne Zeile unserer
#    Daten (df_llm_input) in einen sauberen String, den das LLM 
#    analysieren kann.

# --- 4.1: Definition des SYSTEM_PROMPT ---
#
# Dieser Prompt ist der Kern der Methode. Er instruiert das LLM, 
# die 'zielspalten' im Lichte der 'kontextspalten' semantisch zu bewerten.
# Wir fordern explizit eine JSON-Ausgabe, um die Ergebnisse 
# programmatisch (automatisiert) verarbeiten zu können.

SYSTEM_PROMPT = """
Du bist ein Modell zur kontextuellen Ausreißererkennung in tabellarischen Daten.
Deine Aufgabe ist es, einzelne Datenzeilen zu bewerten.

DATENSATZKONTEXT:
Die Daten stammen aus einem Online-Deal-Forum. Wichtige Spalten sind u.a.:
- 'title': Titel des Deals/Posts
- 'votes': Summe aus Up- und Downvotes (kann positiv oder negativ sein)
- 'source': Anbieter / Händler
- 'replies': Anzahl der Antworten
- 'views': Anzahl der Aufrufe
- 'price': Angebotspreis
- 'saving': ausgewiesene Ersparnis
Negative 'votes'-Werte sind in diesem Kontext grundsätzlich möglich und allein deshalb
kein Ausreißer.

ZIELSPALTEN:
Bewerte ausschließlich die Werte in den Spalten:
['replies', 'views', 'votes'].

KONTEXT:
Nutze alle anderen Spalten der Zeile (z.B. 'title', 'price', 'source', 'thread_category', 'saving')
als Kontextsignale. Sie geben Hinweise auf Thema, Attraktivität, Reichweite, Plattform,
Zielgruppe und Deal-Charakter.

GRUNDIDEE:
Beurteile, ob das beobachtete Engagement in den Zielspalten im Verhältnis zum Kontext
ungewöhnlich ist.

- Wenn das Niveau von 'replies' / 'views' / 'votes' zum Kontext passt,
  behandle den Datenpunkt als nicht auffällig.
- Wenn das Niveau von 'replies' / 'views' / 'votes' im Kontext deutlich unerwartet wirkt
  (z.B. extrem hoch oder extrem niedrig für diese Art Inhalt oder Angebot),
  markiere den Datenpunkt als Ausreißer.

ORIENTIERUNG (NUR ALS DENKHILFE, KEINE HARTE REGELN):

- Weitreichende, wertige, stark nachgefragte oder kontroverse Themen/Deals können hohes
  Engagement ('views', 'replies', 'votes') plausibel erklären.
- Sehr einfache, lokale oder wenig attraktive Inhalte mit extrem hohem Engagement können
  auffällig sein.
- Sehr attraktive oder stark rabattierte Angebote mit nahezu keinem Engagement können
  ebenfalls auffällig sein.
- Ein Wert ist nicht allein deshalb ein Ausreißer, weil er groß, klein oder negativ ist
  entscheide immer im Zusammenspiel mit dem Kontext. Insbesondere negative 'votes' können
  ein normales Bewertungssignal sein.

NUTZE DEIN WISSEN:
Verwende dein allgemeines Weltwissen und dein Verständnis typischer Online-Interaktionen,
um zu entscheiden, ob das Engagement für den gegebenen Kontext deutlich ungewöhnlich
oder im erwartbaren Rahmen ist.

AUSGABEFORMAT (VERPFLICHTEND):
Gib deine Antwort NUR als gültiges JSON-Objekt zurück, ohne zusätzlichen Text.

Das JSON-Format muss exakt wie folgt aussehen:
{
  "is_outlier": true/false,
  "begruendung": "Kurze (1–2 Sätze) Erklärung, warum das Engagement im gegebenen Kontext als unauffällig (false) oder deutlich ungewöhnlich (true) eingestuft wird.",
  "zielspalten_bewertet": ["replies", "views", "votes"]
}
"""
print("SYSTEM_PROMPT (Regelwerk für das LLM) wurde definiert.")

# --- 4.2: Definition der USER_PROMPT Funktion ---
#
# Diese Funktion nimmt eine einzelne Zeile (als Pandas Series) aus 
# unserem 'df_llm_input' DataFrame, konvertiert sie in einen sauberen 
# JSON-String und bettet sie in eine Anweisung für das LLM ein.

def erstelle_user_prompt(daten_zeile):
    """
    Konvertiert eine Pandas Series (Zeile) in einen formatierten 
    JSON-String für den LLM-Prompt.
    """
    try:
        # Konvertiere die Pandas-Zeile in ein Dictionary
        zeilen_dict = daten_zeile.to_dict()
        
        # Konvertiere das Dictionary in einen formatierten JSON-String
        # (indent=2 sorgt für bessere Lesbarkeit, falls wir es debuggen müssen)
        zeilen_json = json.dumps(zeilen_dict, indent=2, ensure_ascii=False)
        
        # Erstelle den finalen Prompt
        user_prompt = f"""
Hier sind die Daten für eine einzelne Zeile im JSON-Format. 
Bewerte diese Zeile gemäß den Anweisungen im SYSTEM_PROMPT.

DATENZEILE:
{zeilen_json}
"""
        return user_prompt

    except Exception as e:
        print(f"FEHLER beim Erstellen des User-Prompts: {e}")
        return None

print("Funktion 'erstelle_user_prompt' (zur Datenformatierung) wurde definiert.")
print("=" * 70)
#
################################################################################

# SCHRITT 5: Testlauf (Probelauf) mit einer einzelnen Zeile
print("\n--- Schritt 5: Testlauf (Probelauf) mit einer einzelnen Zeile ---")

# Methodischer Ansatz:
# Bevor wir das LLM auf alle 1326 Zeilen anwenden (was Zeit und 
# API-Kosten verursacht), führen wir einen Testlauf mit der ERSTEN 
# Zeile (Index 0) des DataFrames 'df_llm_input' durch.
#
# Ziel: Überprüfen, ob die in Schritt 4 definierte Prompt-Strategie 
# (System/User) funktioniert und ob die JSON-Ausgabe (unsere 
# Zwangsvorgabe) korrekt gelesen (geparst) werden kann.
# Dies validiert den Ansatz der "Verarbeitung einzelner Instanzen".

print("Starte Testlauf mit Zeile 0...")

# 1. Testzeile auswählen (Index 0)
test_zeile = df_llm_input.iloc[0]

# 2. User-Prompt für diese Zeile erstellen (mit der Funktion aus Schritt 4)
test_user_prompt = erstelle_user_prompt(test_zeile)

if test_user_prompt is None:
    print("FEHLER: Der Test-User-Prompt konnte nicht erstellt werden.")
    sys.exit("Skript gestoppt.")

# 3. API-Anfrage senden (client wurde in Schritt 2 initialisiert)
try:
    print("Sende Anfrage an Claude API (Modell: claude-sonnet-4-5-20250929)...")
    
    # Modell-Name (gemäß unserer Methodik, Abschnitt 3.3.1)
    # Wichtig: Der spezifische Modellname muss exakt angegeben werden.
    MODELL_NAME = "claude-sonnet-4-5-20250929" 
    
    api_antwort = client.messages.create(
        model=MODELL_NAME,
        system=SYSTEM_PROMPT,
        messages=[
            {"role": "user", "content": test_user_prompt}
        ],
        max_tokens=500,  # Ausreichend Platz für die JSON-Antwort
        temperature=0.0  # Sorgt für deterministische (reproduzierbare) 
                         # Ergebnisse, indem Kreativität unterbunden wird.
    )
    
    # 4. Rohe Antwort extrahieren
    # Die Antwort des LLM befindet sich im 'content[0].text'-Block
    rohe_antwort_text = api_antwort.content[0].text
    print("Rohe Text-Antwort vom LLM erhalten.")

except RateLimitError as e:
    print(f"FEHLER (RateLimitError): API-Ratenlimit überschritten. {e}")
    print("Bitte überprüfen Sie Ihr Anthropic API-Kontingent.")
    sys.exit("Skript gestoppt.")
except APIError as e:
    print(f"FEHLER (APIError): Ein API-Fehler ist aufgetreten. {e}")
    sys.exit("Skript gestoppt.")
except Exception as e:
    print(f"FEHLER (Unbekannt) bei der API-Anfrage: {e}")
    sys.exit("Skript gestoppt.")

# 5. JSON-Antwort parsen (lesen)
# Wir müssen den (manchmal unsauberen) Text in ein sauberes JSON-Objekt umwandeln.
try:
    # Robuste Extraktion: Finde die erste '{' und die letzte '}'
    json_start = rohe_antwort_text.find('{')
    json_ende = rohe_antwort_text.rfind('}') + 1
    
    if json_start == -1 or json_ende == 0:
        raise json.JSONDecodeError("Keine JSON-Struktur ('{' oder '}') gefunden.", 
                                   rohe_antwort_text, 0)
        
    json_text_sauber = rohe_antwort_text[json_start:json_ende]
    
    # Konvertiere den sauberen Text in ein Python-Dictionary
    ergebnis_dict = json.loads(json_text_sauber)
    
    print("\n--- ERGEBNIS DES TESTLAUFS (Geparstes JSON) ---")
    
    # Zeige die Zieldaten, die bewertet wurden:
    print("Originale Zieldaten (Index 0):")
    print(test_zeile[zielspalten].to_dict())
    print("\nLLM-Bewertung:")
    
    # Schöne Ausgabe des Dictionaries (mit Einrückung)
    print(json.dumps(ergebnis_dict, indent=2, ensure_ascii=False))
    
    # 6. Validierung (Prüfung, ob die erwarteten Schlüssel vorhanden sind)
    if 'is_outlier' not in ergebnis_dict or 'begruendung' not in ergebnis_dict:
        print("\nWARNUNG: Die JSON-Antwort enthält nicht die erwarteten \
Schlüssel ('is_outlier', 'begruendung'). Prompt muss evtl. angepasst werden.")
    else:
        print("\nSTATUS: Testlauf erfolgreich. JSON-Format ist korrekt.")

except json.JSONDecodeError as e:
    print(f"FEHLER (JSONDecodeError): Die Antwort des LLM war kein gültiges JSON.")
    print("Dies passiert, wenn das LLM dem SYSTEM_PROMPT (Zwangsausgabe) \
nicht gehorcht hat.")
    print(f"Fehlermeldung: {e}")
    print(f"Empfangener Text (Roh):\n{rohe_antwort_text}")
    sys.exit("Skript gestoppt.")
print("=" * 70)
#
################################################################################

# SCHRITT 6: Ausführung des Experiments auf dem gesamten Datensatz (1326 Zeilen)
print("\n--- Schritt 6: Ausführung des Experiments auf dem gesamten Datensatz ---")

# Methodischer Ansatz:
# Nachdem der Testlauf (Schritt 5) erfolgreich war, wenden wir die 
# Prompt-Strategie nun in einer Schleife (Loop) auf alle 1326 Zeilen 
# des 'df_llm_input'-DataFrames an.
#
# Wir speichern die Ergebnisse (die JSON-Antworten) in einer Liste,
# um sie anschließend in einen neuen DataFrame umzuwandeln.

# WICHTIGER HINWEIS: Dieser Vorgang ist SEHR LANGSAM. 
# Er führt 1326 einzelne API-Anfragen an Claude durch. 
# Dies kann je nach API-Latenz und Ratenlimit 30-60 Minuten dauern.

# Initialisierung der Listen zur Speicherung der Ergebnisse
ergebnis_liste = []
fehler_indizes = [] # Zum Protokollieren von Zeilen, die fehlschlagen

print(f"Starte die Verarbeitung von {len(df_llm_input)} Zeilen...")
print("Dies wird lange dauern. Bitte haben Sie Geduld.")
print("-" * 40)

# Startzeit für die Laufzeitmessung des gesamten LLM-Experiments
start_zeit = time.time()

# --- START DER SCHLEIFE (LOOP) ---
# .itertuples() ist schneller als .iterrows() für Pandas-Schleifen
for zeile in df_llm_input.itertuples():
    
    # Zeige einen Fortschrittsindikator alle 25 Zeilen
    if zeile.Index % 25 == 0 and zeile.Index > 0:
        print(f"  ...verarbeite Zeile {zeile.Index} von {len(df_llm_input)}")
        # (Optional: Eine kleine Pause, um API-Ratenlimits zu schonen)
        # time.sleep(1) 

    # 1. User-Prompt erstellen
    # (Wir müssen das 'zeile'-Tupel in eine Series umwandeln, 
    # damit unsere Funktion 'erstelle_user_prompt' funktioniert)
    zeilen_series = zeile._asdict()
    # Entferne den 'Index', da er nicht Teil der Kontextspalten ist
    zeilen_series.pop('Index', None) 
    
    # Konvertiere zurück zu einer Pandas Series (wie in Schritt 5 erwartet)
    zeilen_series_pd = pd.Series(zeilen_series)
    
    user_prompt = erstelle_user_prompt(zeilen_series_pd)
    
    if user_prompt is None:
        print(f"FEHLER (Zeile {zeile.Index}): User-Prompt konnte nicht erstellt werden. \
Zeile wird übersprungen.")
        fehler_indizes.append(zeile.Index)
        continue # Nächste Zeile

    # 2. API-Anfrage senden
    try:
        api_antwort = client.messages.create(
            model=MODELL_NAME, # (Definiert in Schritt 5)
            system=SYSTEM_PROMPT,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=500,
            temperature=0.0
        )
        
        rohe_antwort_text = api_antwort.content[0].text
        
        # 3. JSON-Antwort parsen (wie in Schritt 5)
        json_start = rohe_antwort_text.find('{')
        json_ende = rohe_antwort_text.rfind('}') + 1
        json_text_sauber = rohe_antwort_text[json_start:json_ende]
        ergebnis_dict = json.loads(json_text_sauber)
        
        # 4. Ergebnisse speichern
        # Füge den originalen Index zur Nachverfolgung hinzu
        ergebnis_dict['original_index'] = zeile.Index
        ergebnis_liste.append(ergebnis_dict)

    except RateLimitError as e:
        print(f"FEHLER (RateLimitError) bei Zeile {zeile.Index}: {e}")
        print("Pausiere für 30 Sekunden und versuche es erneut...")
        time.sleep(30)
        # (Hier könnte man einen 'retry'-Mechanismus einbauen, 
        # aber zur Vereinfachung überspringen wir die Zeile)
        fehler_indizes.append(zeile.Index)
        continue
        
    except json.JSONDecodeError as e:
        print(f"FEHLER (JSONDecodeError) bei Zeile {zeile.Index}: LLM gab kein \
gültiges JSON zurück. Zeile wird übersprungen.")
        print(f"Empfangener Text (Roh): {rohe_antwort_text[:100]}...") # Zeige nur die ersten 100 Zeichen
        fehler_indizes.append(zeile.Index)
        continue
        
    except Exception as e:
        print(f"FEHLER (Unbekannt) bei Zeile {zeile.Index}: {e}. \
Zeile wird übersprungen.")
        fehler_indizes.append(zeile.Index)
        continue

# --- ENDE DER SCHLEIFE ---

end_zeit = time.time()
laufzeit_sek = end_zeit - start_zeit
durchschnitt_pro_zeile = laufzeit_sek / len(df_llm_input)

print("-" * 40)
print(f"LLM-Gesamtlaufzeit: {laufzeit_sek:.2f} Sekunden "
      f"(≈ {laufzeit_sek/60:.2f} Minuten).")
print(f"Durchschnittliche Laufzeit pro Zeile: {durchschnitt_pro_zeile:.2f} Sekunden.")

print("Verarbeitung aller Zeilen abgeschlossen.")
print(f"Erfolgreich verarbeitete Zeilen: {len(ergebnis_liste)}")
print(f"Fehlgeschlagene/Übersprungene Zeilen: {len(fehler_indizes)}")
if len(fehler_indizes) > 0:
    print(f"Fehlerhafte Indizes: {fehler_indizes}")
print("-" * 40)
#
################################################################################

# SCHRITT 7: Ergebnisse verarbeiten und in DataFrame umwandeln
print("\n--- Schritt 7: Ergebnisse verarbeiten und in DataFrame umwandeln ---")

# Die 'ergebnis_liste' (aus der vorherigen Zelle, Schritt 6) 
# ist noch im Arbeitsspeicher vorhanden.

if len(ergebnis_liste) == 0:
    print("FEHLER: Keine Ergebnisse zum Verarbeiten vorhanden. \
Möglicherweise ist die vorherige Zelle (Schritt 6) fehlgeschlagen.")
    sys.exit("Skript gestoppt.")

# Konvertiere die Liste von Dictionaries (JSON-Antworten) in einen Pandas DataFrame
df_ergebnisse_llm = pd.DataFrame(ergebnis_liste)

# Setze den 'original_index' als Hauptindex des neuen DataFrames,
# damit wir die Ergebnisse leicht den Originaldaten zuordnen können.
df_ergebnisse_llm = df_ergebnisse_llm.set_index('original_index')

# Zeige die ersten 5 Ergebnisse (Entscheidungen + Begründungen)
print("Verarbeitung abgeschlossen. Hier sind die ersten 5 LLM-Bewertungen:")
print(df_ergebnisse_llm.head())
print("=" * 70)

#
################################################################################

# SCHRITT 8: Ergebnisse speichern und Zusammenfassung
print("\n--- Schritt 8: Ergebnisse speichern und Zusammenfassung ---")

os.makedirs('ergebnisse', exist_ok=True)

# Methodischer Ansatz (gemäß ZWECK-Schritt 6):
# Wir speichern ZWEI Dateien, um die quantitative UND qualitative 
# Analyse (Forschungsfrage 4) zu ermöglichen.

# 1. Definieren der Dateipfade
ergebnis_pfad_komplett = 'ergebnisse/1.5_llm_komplette_ergebnisse.csv'
ergebnis_pfad_indizes = 'ergebnisse/1.5_llm_ausreisser_indizes.csv'

# 2. Speichern der KOMPLETTEN Ergebnisse (inkl. Begründungen)
#    Dies ist unsere qualitative Datengrundlage für Forschungsfrage 4.
try:
    df_ergebnisse_llm.to_csv(ergebnis_pfad_komplett, index=True)
    print(f"Qualitative Ergebnisse (inkl. Begründungen) gespeichert in: \
'{ergebnis_pfad_komplett}'")
except Exception as e:
    print(f"FEHLER beim Speichern der kompletten Ergebnisse: {e}")


# 3. Speichern der INDIZES (für den quantitativen Vergleich)
#    Wir filtern nur die Zeilen, die das LLM als 'true' (Ausreißer) 
#    markiert hat, um eine Indexliste zu erhalten, die mit 
#    IQR, LOF etc. vergleichbar ist.
try:
    # Filtere den DataFrame, wo 'is_outlier' == True ist
    df_nur_ausreisser = df_ergebnisse_llm[
        df_ergebnisse_llm['is_outlier'] == True
    ]
    
    # Speichere nur den Index dieses gefilterten DataFrames
    df_nur_ausreisser.index.to_series().to_csv(
        ergebnis_pfad_indizes, 
        index=False, 
        header=['Ausreisser_Index']
    )
    print(f"Quantitative Indizes (nur Ausreißer) gespeichert in: \
'{ergebnis_pfad_indizes}'")
    
    # Zähle das Ergebnis für die Zusammenfassung
    anzahl_gefundener_ausreisser = len(df_nur_ausreisser)

except KeyError:
    print("FEHLER: Spalte 'is_outlier' nicht in den LLM-Ergebnissen gefunden. \
Überprüfen Sie den SYSTEM_PROMPT.")
    anzahl_gefundener_ausreisser = 0
except Exception as e:
    print(f"FEHLER beim Speichern der Indizes: {e}")
    anzahl_gefundener_ausreisser = -1 # Fehlercode

print("=" * 70)
#
################################################################################

# SCHRITT 9: Zusammenfassung (Protokoll)
print("\n--- SCHRITT 9: Zusammenfassung EXPERIMENT 1.5 (LLM) ---")
print(f"Methode:           Moderne KI: LLM (Claude 4.5 Sonnet)")
print(f"Zieldaten:         {dateipfad} (Shape: {df_schmutzig.shape})")
print(f"Zielspalten:       {zielspalten}")
print(f"Kontextspalten:    {len(kontextspalten)} Spalten")
print(f"Verarbeitete Zeilen: {len(ergebnis_liste)} / {len(df_llm_input)}")
print("-" * 40)
print(f"ERGEBNIS (COUNT):  {anzahl_gefundener_ausreisser} einzigartige \
Ausreißerzeilen identifiziert.")
print("=" * 70)
print("Die Evaluierung dieser Indizes erfolgt gemäß der quantitativen "
      "Evaluationsstrategie (Abschnitt 3.4.1); die Analyse der LLM-Begründungen "
      "gemäß der qualitativen Evaluationsstrategie (Abschnitt 3.4.2) "
      "in einem separaten Evaluierungs-Skript.")
print("=" * 70)
print("=== ENDE SKRIPT 06 ===")

Lade notwendige Bibliotheken (Pandas, Numpy, Matplotlib, OS, sys, json)...
Anthropic (Claude) Bibliothek erfolgreich geladen.
Alle Bibliotheken sind bereit.
--- Schritt 2: 'Schmutzigen' Rohdatensatz laden & API-Client initialisieren ---
Datensatz geladen: rfd_main.csv
Dimensionen (Zeilen, Spalten): (1326, 15)
----------------------------------------
Anthropic (Claude) API-Client erfolgreich initialisiert.

--- Schritt 3: Definition der Ziel- und Kontextspalten ---
Zielspalten (zu bewerten): ['replies', 'views', 'votes']
Kontextspalten (zur Analyse): 10 Spalten werden an das LLM gesendet.

Bereite LLM-Input-Daten vor (Konvertiere NaN zu leeren Strings)...
Vorbereitung abgeschlossen. Datenstruktur für LLM-Input:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326 entries, 0 to 1325
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   title            1326 non-null   object
 1   price            1326 non-null   